In [1]:
import pandas as pd
import numpy as np
# from expenses import * 
# from revenues import * 
# from other import *
# from model import *
from model import * 



In [2]:
# base case
a = FinancialModel()
a.run_base_case()
print(a.roe)
# a.df_out_format



3.8770638219136027


### exchange rate fluctiations affecting EPC contract price 

In [5]:
# TODO add to inputs! 

current_exchange = 1.30 # currently = 1.30 eur => us 

# price has fluctuated from 0.95 to 1.39 over past 10 years 
# https://www.xe.com/currencycharts/?from=EUR&to=USD&view=10Y
# lets do the full range, and then some, +/- 0.15
rates = np.round(np.arange(0.80, 1.55, 0.05),2)

# 0.01 increase in exchange rate => EPC contract price increases / decreases by 1,670,000
epc_fluct = 1670e3/0.01

devs = np.round(rates - current_exchange,3)
dev_cash = np.round(devs * epc_fluct,3)

print(rates, devs, dev_cash)

[0.8  0.85 0.9  0.95 1.   1.05 1.1  1.15 1.2  1.25 1.3  1.35 1.4  1.45
 1.5 ] [-0.5  -0.45 -0.4  -0.35 -0.3  -0.25 -0.2  -0.15 -0.1  -0.05  0.    0.05
  0.1   0.15  0.2 ] [-83500000. -75150000. -66800000. -58450000. -50100000. -41750000.
 -33400000. -25050000. -16700000.  -8350000.         0.   8350000.
  16700000.  25050000.  33400000.]


In [11]:
roe_dict = {}
for cash_delta, dev in zip(dev_cash, devs):
    b = FinancialModel()
    curr_epc_val = b.epc_data["EPC Cost"]
    
    new_val = curr_epc_val +cash_delta
    
    roe_dict[dev] = (b.eval_epc_currency_sens(new_val), cash_delta)

roe_dict

{-0.5: (4.231265807881294, -83500000.0),
 -0.45: (4.195845609284525, -75150000.0),
 -0.4: (4.160425410687756, -66800000.0),
 -0.35: (4.125005212090986, -58450000.0),
 -0.3: (4.089585013494218, -50100000.0),
 -0.25: (4.054164814897448, -41750000.0),
 -0.2: (4.018744616300679, -33400000.0),
 -0.15: (3.98332441770391, -25050000.0),
 -0.1: (3.9479042191071407, -16700000.0),
 -0.05: (3.9124840205103717, -8350000.0),
 0.0: (3.8770638219136027, 0.0),
 0.05: (3.841643623316833, 8350000.0),
 0.1: (3.806223424720064, 16700000.0),
 0.15: (3.770803226123295, 25050000.0),
 0.2: (3.735383027526526, 33400000.0)}

In [9]:
roe_dict

{0.8: (4.231265807881294, 0.8),
 0.85: (4.195845609284525, 0.85),
 0.9: (4.160425410687756, 0.9),
 0.95: (4.125005212090986, 0.95),
 1.0: (4.089585013494218, 1.0),
 1.05: (4.054164814897448, 1.05),
 1.1: (4.018744616300679, 1.1),
 1.15: (3.98332441770391, 1.15),
 1.2: (3.9479042191071407, 1.2),
 1.25: (3.9124840205103717, 1.25),
 1.3: (3.8770638219136027, 1.3),
 1.35: (3.841643623316833, 1.35),
 1.4: (3.806223424720064, 1.4),
 1.45: (3.770803226123295, 1.45),
 1.5: (3.735383027526526, 1.5)}

In [4]:
b = FinancialModel()
b.eval_epc_currency_sens(10)

5.807146457521744